# ReAct Agent with LangGraph: Academic Paper Analysis Agent

This notebook implements a LangGraph ReAct agent that can analyze academic papers (PDF files) using LangGraph and Amazon Nova. The agent has specific tools to:
1. Summarize papers
2. Extract research questions
3. Extract key results
4. Identify research gaps

## Setup and Dependencies

In [1]:
# Install required packages
!pip install -q langchain langchain-aws langgraph pypdf boto3

In [2]:
import os
import json
import boto3
import pypdf
from typing import Dict, List, Any, Optional, Union, Literal, TypedDict
from pprint import pprint
from pydantic import BaseModel, Field

# LangChain and LangGraph imports
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage, FunctionMessage, ToolMessage
#from langchain_aws import BedrockChat
from langchain.tools import BaseTool, tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import create_react_agent

Setting model_id and AWS region

In [3]:
region = "us-east-1"
model_id = "us.amazon.nova-premier-v1:0"

## PDF Extraction Utility

In [4]:
def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file"""
    with open(pdf_path, 'rb') as file:
        pdf_reader = pypdf.PdfReader(file)
        text = ""
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
    return text

def get_paper_sections(text):
    """Simple function to split paper into sections based on common headers"""
    # This is a very simplified approach - real implementation would be more sophisticated
    sections = {}
    
    # Common section headers in academic papers
    section_markers = [
        "abstract", "introduction", "related work", "background", "methodology", 
        "methods", "experimental setup", "results", "discussion", 
        "conclusion", "future work", "references"
    ]
    
    # Split by common section headers (very naive approach)
    lines = text.split('\n')
    current_section = "preamble"
    sections[current_section] = []
    
    for line in lines:
        lower_line = line.lower().strip()
        if any(lower_line.startswith(marker) or lower_line == marker for marker in section_markers):
            current_section = lower_line
            sections[current_section] = []
        else:
            sections[current_section].append(line)
    
    # Convert lists of strings to single strings
    for section in sections:
        sections[section] = "\n".join(sections[section])
    
    return sections

## Amazon Bedrock Setup

In [5]:
# AWS Bedrock setup
# You need to have AWS credentials configured
# Note: Replace with your AWS credentials and region if you are not using IAM role
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=region,  # Change to your preferred region
    # Use the lines below if you are not using IAM ROLE - remember to never share your credentials or save them in code repos
    # aws_access_key_id=os.environ.get('AWS_ACCESS_KEY_ID'),
    # aws_secret_access_key=os.environ.get('AWS_SECRET_ACCESS_KEY')
)

# Create Bedrock LLM
llm = ChatBedrock(
    model_id=model_id,
    client=bedrock_client,
    model_kwargs={
        "temperature": 0.1,
        "max_tokens": 4000,
    }
)

In [6]:
# Testing our setup
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg.content

"J'adore la programmation."

## Agent State and Paper Context

In [7]:
class PaperContext:
    """Class to hold the context of the paper being analyzed"""
    def __init__(self, paper_path=None):
        self.paper_path = paper_path
        self.full_text = ""
        self.sections = {}
        self.title = ""
        self.authors = []
        self.abstract = ""
        
        if paper_path and os.path.exists(paper_path):
            self.load_paper(paper_path)
    
    def load_paper(self, paper_path):
        """Load and parse a paper from a PDF file"""
        self.paper_path = paper_path
        self.full_text = extract_text_from_pdf(paper_path)
        self.sections = get_paper_sections(self.full_text)
        
        # Extract basic metadata (very naive approach)
        lines = self.full_text.split('\n')
        self.title = lines[0] if lines else ""
        
        # Get abstract if it exists
        if "abstract" in self.sections:
            self.abstract = self.sections["abstract"]
    
    def get_section(self, section_name):
        """Get a specific section of the paper"""
        # Case-insensitive section lookup
        for key in self.sections:
            if section_name.lower() in key.lower():
                return self.sections[key]
        return ""

# Create a global paper context object
paper_context = PaperContext()

## Define Agent Tools

In [8]:
@tool
def load_paper(file_path: str) -> str:
    """Load an academic paper from a PDF file path"""
    if not os.path.exists(file_path):
        return f"Error: File {file_path} does not exist."
    
    try:
        paper_context.load_paper(file_path)
        return f"Successfully loaded paper: {paper_context.title}. Paper has {len(paper_context.sections)} sections."
    except Exception as e:
        return f"Error loading paper: {str(e)}"

@tool
def get_paper_summary() -> str:
    """Generate a comprehensive summary of the loaded academic paper"""
    if not paper_context.full_text:
        return "No paper loaded. Please load a paper first."
    
    prompt = f"""
    Create a comprehensive summary of this academic paper. Focus on the main contributions,
    methodology, and key findings. The summary should be around 500 words.
    
    Paper Title: {paper_context.title}
    Abstract: {paper_context.abstract}
    
    Paper content:
    {paper_context.full_text[:10000]}  # Take the first 10000 chars for summary
    """
    
    response = llm.invoke(prompt)
    return response.content

@tool
def extract_research_questions() -> List[str]:
    """Extract and list the research questions addressed in the paper"""
    if not paper_context.full_text:
        return "No paper loaded. Please load a paper first."
    
    # Get intro and potentially methods sections for research questions
    intro = paper_context.get_section("introduction")
    
    prompt = f"""
    Identify the main research questions or hypotheses addressed in this academic paper.
    Extract them as an explicit list. If research questions are not explicitly stated,
    infer them from the goals and objectives of the paper.
    
    Paper Title: {paper_context.title}
    Abstract: {paper_context.abstract}
    Introduction: {intro[:5000]}  # Use intro section if available
    
    Format your response as a JSON list of research questions, like this:
    ```json
    [
      "Research question 1",
      "Research question 2"
    ]
    ```
    """
    
    response = llm.invoke(prompt)
    
    # Parse the response to extract the JSON list
    try:
        # Look for JSON in triple backticks
        import re
        json_match = re.search(r'```(?:json)?\n(.+?)\n```', response.content, re.DOTALL)
        if json_match:
            questions = json.loads(json_match.group(1))
        else:
            # Try to find a JSON array directly
            json_match = re.search(r'\[.+\]', response.content, re.DOTALL)
            if json_match:
                questions = json.loads(json_match.group(0))
            else:
                # If all else fails, return the raw response
                return response.content
        
        return questions
    except Exception as e:
        # If JSON parsing fails, return the raw text
        return response.content

In [9]:
@tool
def extract_key_results() -> Dict[str, Any]:
    """Extract the key results and findings from the paper"""
    if not paper_context.full_text:
        return "No paper loaded. Please load a paper first."
    
    # Get results and discussion sections
    results = paper_context.get_section("results")
    discussion = paper_context.get_section("discussion")
    
    prompt = f"""
    Extract the key results and findings from this academic paper.
    Focus on quantitative results, main findings, and their significance.
    
    Paper Title: {paper_context.title}
    Abstract: {paper_context.abstract}
    Results section: {results[:5000]}
    Discussion section: {discussion[:5000]}
    
    Format your response as a JSON object with these keys:
    1. main_findings: A list of the main findings (each 1-2 sentences)
    2. quantitative_results: Key numerical results and metrics
    3. significance: The significance of these findings in the field
    
    Example format:
    ```json
    {{
      "main_findings": [
        "Finding 1",
        "Finding 2"
      ],
      "quantitative_results": {{
        "metric1": "value1",
        "metric2": "value2"
      }},
      "significance": "Description of why these results matter"
    }}
    ```
    """
    
    response = llm.invoke(prompt)
    
    # Parse the response to extract the JSON object
    try:
        # Look for JSON in triple backticks
        import re
        json_match = re.search(r'```(?:json)?\n(.+?)\n```', response.content, re.DOTALL)
        if json_match:
            results_dict = json.loads(json_match.group(1))
        else:
            # Try to find a JSON object directly
            json_match = re.search(r'\{.+\}', response.content, re.DOTALL)
            if json_match:
                results_dict = json.loads(json_match.group(0))
            else:
                # If all else fails, return the raw response
                return response.content
        
        return results_dict
    except Exception as e:
        # If JSON parsing fails, return the raw text
        return response.content

@tool
def identify_research_gaps() -> List[str]:
    """Identify research gaps and potential future work based on the paper"""
    if not paper_context.full_text:
        return "No paper loaded. Please load a paper first."
    
    # Get discussion, conclusion, and future work sections
    discussion = paper_context.get_section("discussion")
    conclusion = paper_context.get_section("conclusion")
    future_work = paper_context.get_section("future work")
    
    prompt = f"""
    Identify research gaps and potential future work based on this academic paper.
    Look for:
    1. Limitations explicitly mentioned by the authors
    2. Areas where the authors suggest future work
    3. Implicit gaps that could be addressed in future research
    
    Paper Title: {paper_context.title}
    Abstract: {paper_context.abstract}
    Discussion: {discussion[:3000]}
    Conclusion: {conclusion[:3000]}
    Future Work: {future_work[:3000]}
    
    Format your response as a JSON list of research gaps, with each gap including a description
    and whether it was explicitly mentioned by the authors or inferred.
    
    Example format:
    ```json
    [
      {{
        "description": "Description of research gap 1",
        "type": "explicit" or "implicit",
        "potential_direction": "Potential research direction to address this gap"
      }},
      {{
        "description": "Description of research gap 2",
        "type": "explicit" or "implicit",
        "potential_direction": "Potential research direction to address this gap"
      }}
    ]
    ```
    """
    
    response = llm.invoke(prompt)
    
    # Parse the response to extract the JSON list
    try:
        # Look for JSON in triple backticks
        import re
        json_match = re.search(r'```(?:json)?\n(.+?)\n```', response.content, re.DOTALL)
        if json_match:
            gaps = json.loads(json_match.group(1))
        else:
            # Try to find a JSON array directly
            json_match = re.search(r'\[.+\]', response.content, re.DOTALL)
            if json_match:
                gaps = json.loads(json_match.group(0))
            else:
                # If all else fails, return the raw response
                return response.content
        
        return gaps
    except Exception as e:
        # If JSON parsing fails, return the raw text
        return response.content

@tool
def get_section_content(section_name: str) -> str:
    """Get the content of a specific section from the paper"""
    if not paper_context.full_text:
        return "No paper loaded. Please load a paper first."
    
    content = paper_context.get_section(section_name)
    if not content:
        return f"Section '{section_name}' not found in the paper."
    
    return content

# Combine all tools
tools = [load_paper, get_paper_summary, extract_research_questions, 
         extract_key_results, identify_research_gaps, get_section_content]

## Define Agent State

In [10]:
class AgentState(TypedDict):
    user_input: str
    messages: List[Union[HumanMessage, AIMessage, SystemMessage, FunctionMessage]]
    paper_loaded: bool
    paper_title: str
    paper_summary: Optional[str]
    research_questions: Optional[List[str]]
    key_results: Optional[Dict[str, Any]]
    research_gaps: Optional[List[Dict[str, str]]]
    final_report: Optional[str]

## Create the Agent with LangGraph

In [11]:
# Define system prompt for the academic paper analysis agent
system_prompt = """You are an academic research assistant specializing in analyzing scientific papers.
Your purpose is to help researchers understand papers by extracting key information like:
- Summarizing the paper's content
- Identifying the main research questions
- Extracting key results and findings
- Identifying research gaps and future directions

You have access to tools that can:
1. Load academic papers from PDF files
2. Generate comprehensive summaries
3. Extract research questions
4. Extract key results and findings
5. Identify research gaps and potential future work
6. Get content from specific sections of the paper

Think step-by-step about what tools you need to use to fulfill the user's request.
Always start by loading the paper if the user has provided a file path.
Use academic language and be precise in your responses.
"""

# Create prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    MessagesPlaceholder(variable_name="messages"),
])

# Create the agent using LangGraph
paper_agent = create_react_agent(
    model=llm, 
    tools=tools, 
    prompt=prompt
)

In [12]:
# Default state
def create_initial_state(user_input: str) -> AgentState:
    return {
        "user_input": user_input,
        "messages": [HumanMessage(content=user_input)],
        "paper_loaded": False,
        "paper_title": "",
        "paper_summary": None,
        "research_questions": None,
        "key_results": None,
        "research_gaps": None,
        "final_report": None
    }

# Function to process the agent's action
def run_agent(state: AgentState) -> dict:
    """Run the agent on the current state and update with results"""
    messages = state["messages"]
    response = paper_agent.invoke({"messages": messages})
    
    updated_state = state.copy()
    updated_state["messages"] = messages + [response]
    
    # Check if the most recent message is a tool response
    if isinstance(response, ToolMessage):
        # Store tool outputs in appropriate state fields
        if response.name == "get_paper_summary":
            updated_state["paper_summary"] = response.content
        elif response.name == "extract_research_questions":
            try:
                # May need to parse JSON if the content is a string representation of JSON
                updated_state["research_questions"] = response.content
            except:
                pass
        elif response.name == "extract_key_results":
            try:
                updated_state["key_results"] = response.content
            except:
                pass
        elif response.name == "identify_research_gaps":
            try:
                updated_state["research_gaps"] = response.content
            except:
                pass
    
    # Update paper information if loaded
    if not state["paper_loaded"] and paper_context.full_text:
        updated_state["paper_loaded"] = True
        updated_state["paper_title"] = paper_context.title
    
    return updated_state

# Build the workflow graph
workflow = StateGraph(AgentState)
workflow.add_node("agent", run_agent)
workflow.set_entry_point("agent")
workflow.add_conditional_edges(
    "agent",
    lambda state: END if not any(hasattr(msg, "tool_calls") and msg.tool_calls 
                                for msg in state["messages"][-1:]) else "agent"
)
app = workflow.compile()

In [13]:
def run_paper_agent(query: str):
    """Run the paper agent with a custom query and return the final response"""
    initial_state = create_initial_state(query)
    response = app.invoke(initial_state)
    #debug
    print("RAW RESPONSE:")
    pprint(response)
    
    # Extract the final message content
    final_message = None
    thinking_contents = []
    
    # Parse the nested message structure
    for message_item in response["messages"]:
        # Check if this item contains a nested messages list
        if isinstance(message_item, dict) and "messages" in message_item:
            for sub_message in message_item["messages"]:
                # Look for AIMessages with content as a list
                if hasattr(sub_message, "content") and isinstance(sub_message.content, list):
                    # Extract thinking content
                    for content_item in sub_message.content:
                        if isinstance(content_item, dict) and content_item.get('type') == 'text' and '<thinking>' in content_item.get('text', ''):
                            thinking_contents.append(content_item.get('text'))
                
                # If this is a string content, it might be the final response
                if hasattr(sub_message, "content") and isinstance(sub_message.content, str):
                    final_message = sub_message.content
        
        # Also check direct messages
        if hasattr(message_item, "content") and isinstance(message_item.content, list):
            for content_item in message_item.content:
                if isinstance(content_item, dict) and content_item.get('type') == 'text' and '<thinking>' in content_item.get('text', ''):
                    thinking_contents.append(content_item.get('text'))
        
        if hasattr(message_item, "content") and isinstance(message_item.content, str):
            final_message = message_item.content
    return {
        "paper_loaded": response["paper_loaded"],
        "paper_title": response["paper_title"],
        "final_response": final_message,
        "thinking_contents": thinking_contents
    }

## Full Paper Analysis Function

In [14]:
# here is our paper, update the path if you are using a different directory.
paper_path = "data/a-shopping-agent-for-addressing-subjective-product-needs.pdf"

Lets test our tools with different queries.

In [15]:
# Example 1: Just extract research questions
query1 = f"what research questions it addresses for the paper? {paper_path}"

# Our agent will first return the raw response
agent_response = run_paper_agent(query1)

RAW RESPONSE:
{'final_report': None,
 'key_results': None,
 'messages': [HumanMessage(content='what research questions it addresses for the paper? data/a-shopping-agent-for-addressing-subjective-product-needs.pdf', additional_kwargs={}, response_metadata={}, id='23c97460-2bde-42f4-800b-c38697a2e4cc'),
              {'messages': [HumanMessage(content='what research questions it addresses for the paper? data/a-shopping-agent-for-addressing-subjective-product-needs.pdf', additional_kwargs={}, response_metadata={}, id='23c97460-2bde-42f4-800b-c38697a2e4cc'),
                            AIMessage(content=[{'type': 'text', 'text': '<thinking>\nTo extract the research questions from the provided paper, I need to first load the paper and then use the tool designed to extract research questions.\n</thinking>\n'}, {'type': 'tool_use', 'name': 'load_paper', 'input': {'file_path': 'data/a-shopping-agent-for-addressing-subjective-product-needs.pdf'}, 'id': 'tooluse_wuAdHutXQkWHmHqKpfvogw'}, {'type'

Lets print the key points of the response just to make it easy to visualise.

In [17]:
# Here are the key points from the response.

pprint(agent_response)

{'final_response': 'The paper addresses the following research questions:\n'
                   '\n'
                   '1. How can an autonomous shopping agent effectively '
                   'address subjective product needs in e-commerce scenarios?\n'
                   '2. What are the key facets of subjective product needs '
                   'that the agent should consider?\n'
                   '3. How can the agent reduce user effort and cognitive load '
                   'during the product exploration and selection process?\n'
                   '4. What computational, conversational, browsing, and '
                   'generative actions can the agent perform to streamline the '
                   'shopping experience?\n'
                   '5. How can customer reviews be utilized to enhance the '
                   "agent's ability to meet subjective needs?\n"
                   '6. What is the impact of the agent on the time saved and '
                   'decision fati

Check the thinking block:
`To extract the research questions from the paper, I need to first load the paper and then use the tool designed to extract research questions.`

As you can see above, our agent invoked our model 3 times:
1. Executed `load_paper` tool to load the paper.
2. Executed `extract_research_questions` tool to extract research questions.
3. Generated the final response.

Time to send another couple of queries, we want to check the key results/findings and also a comprehensive report on the paper including summary, research questions, key findings and research gaps.

In [18]:
# Example 2: Extract key results findings
query2 = f"What are the key results and findings? {paper_path}"

# Our agent is starting with clean state.
agent_response = run_paper_agent(query2)

RAW RESPONSE:
{'final_report': None,
 'key_results': None,
 'messages': [HumanMessage(content='What are the key results and findings? data/a-shopping-agent-for-addressing-subjective-product-needs.pdf', additional_kwargs={}, response_metadata={}, id='6f39f51c-9c42-4350-a007-eabf8399db04'),
              {'messages': [HumanMessage(content='What are the key results and findings? data/a-shopping-agent-for-addressing-subjective-product-needs.pdf', additional_kwargs={}, response_metadata={}, id='6f39f51c-9c42-4350-a007-eabf8399db04'),
                            AIMessage(content=[{'type': 'text', 'text': '<thinking>\nTo extract the key results and findings from the paper, I need to first load the paper and then use the tool designed to extract key results and findings.\n</thinking>\n'}, {'type': 'tool_use', 'name': 'load_paper', 'input': {'file_path': 'data/a-shopping-agent-for-addressing-subjective-product-needs.pdf'}, 'id': 'tooluse_qn87slJBTwm-sMehYHtf8g'}, {'type': 'tool_use', 'name': '

In [23]:
# Here are the key points from the response.
pprint(agent_response)

{'final_response': 'The key results and findings from the paper "A Shopping '
                   'Agent for Addressing Subjective Product Needs" are as '
                   'follows:\n'
                   '\n'
                   '### Main Findings:\n'
                   '1. **Effectiveness in Reducing User Effort**: The proposed '
                   'shopping agent effectively reduces user effort by '
                   'autonomously executing various actions such as vagueness '
                   'detection, subjective product needs extraction, and '
                   'browsing actions.\n'
                   '2. **Time Savings**: The agent can save users significant '
                   'time by processing and ranking numerous reviews, thereby '
                   'reducing decision fatigue and promoting the exploration of '
                   'a wider range of products.\n'
                   '\n'
                   '### Quantitative Results:\n'
                   '- **Time Saved**: 

Check the thinking block again:
`To extract the key results and findings from the paper, I need to first load the paper and then use the tool designed to extract key results and findings.`

This time the agent executed the following steps:
1. `load_paper` tool again (remember we are not keeping history so we can see the whole workflow).
2. `main_findings` tool.
3. Generated the final response.

In [ ]:
# Example 3: Generatea comprehensive report, including a summary, research questions, key findings and research gaps.
query3 = f"Please generate a comprehensive report on the paper, including a summary, research questions, key findings, and research gaps. Here is the paper: {paper_path}"

agent_response = run_paper_agent(query3)

In [ ]:
# Here are the key points from the response.
pprint(agent_response)

We can see a few other tools in the output above, the thinking block was more complex as well. Looking at the thinking block we can see the agent planing:
```
To generate a comprehensive report on the paper, I need to follow these steps:
1. Load the paper from the provided file path.
2. Generate a comprehensive summary of the paper.
3. Extract the research questions addressed in the paper.
4. Extract the key results and findings from the paper.\n5. Identify research gaps and potential future work based on the paper.\n\nI will start by loading the paper and then proceed with the other steps.
```